2015 Relation Classification via Recurrent Neural Network
Dongxu Zhang, Dong Wang
1 layer of BiRNN
h = max(ht_fw + ht_bw, for t in 0:length)

In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import time
print(tf.__version__)

1.10.0


In [2]:
import mnistdata
mnist = mnistdata.read_data_sets("data/fashion", one_hot=True, reshape=False)

Successfully unzipped train-images-idx3-ubyte.gz
Successfully unzipped train-labels-idx1-ubyte.gz
Successfully unzipped t10k-images-idx3-ubyte.gz
Successfully unzipped t10k-labels-idx1-ubyte.gz


In [3]:
print(mnist.train.images.shape)
print(mnist.test.labels.shape)

(60000, 28, 28, 1)
(10000, 10)


In [4]:
learning_rate = 1e-4
batch_size = 100
display_step = 1000
test_size = mnist.test.labels.shape[0]

n_input = 28
n_steps=  28
n_hidden = 128
n_classes = 10

In [5]:
X = tf.placeholder(tf.float32, [None, n_steps, n_input])
Y_ = tf.placeholder(tf.float32, [None, n_classes])

W = tf.Variable(tf.truncated_normal([n_hidden, n_classes], stddev=0.1))
b = tf.Variable(tf.truncated_normal([n_classes], stddev=0.1))

In [6]:
def BiRNN(x, w, b):
    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, n_input])
    x = tf.split(x, n_steps)

    fw_cell = tf.nn.rnn_cell.LSTMCell(n_hidden, forget_bias=1.0)
    bw_cell = tf.nn.rnn_cell.LSTMCell(n_hidden, forget_bias=1.0)
    out, _, _ = tf.contrib.rnn.static_bidirectional_rnn(fw_cell, bw_cell, x, dtype=tf.float32)
    
    out = tf.stack(out)
    h = out[:, :, 0:n_hidden] + out[:, :, n_hidden:2*n_hidden]
    h_max = tf.reduce_max(h, 0)
    
    pred = tf.nn.softmax(tf.matmul(h_max, w) + b)

    return pred

In [7]:
Y = BiRNN(X, W, b)

In [8]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Y, labels=Y_))

correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [9]:
# mnist
start = time.clock()
test_X = mnist.test.images
test_X = test_X.reshape([test_size, n_steps, n_input])
test_Y = mnist.test.labels
for i in range(10000):
    batch_X, batch_Y = mnist.train.next_batch(batch_size)
    batch_X = batch_X.reshape([batch_size, n_steps, n_input])
    if i % display_step == 0:
        train_accuracy = accuracy.eval(session=sess, feed_dict={X:batch_X, Y_:batch_Y})
        test_accuracy = accuracy.eval(session=sess, feed_dict={X:test_X, Y_:test_Y})
        print("step %5d, train_accuracy = %.4g %% test_accuracy = %.4g %%" % (i, train_accuracy*100, test_accuracy*100))
    train_step.run(session=sess, feed_dict={X:batch_X, Y_:batch_Y})
print("training time: ", time.clock()-start, " s")

step     0, train_accuracy = 6 % test_accuracy = 9 %
step  1000, train_accuracy = 87 % test_accuracy = 83.12 %
step  2000, train_accuracy = 88 % test_accuracy = 94.59 %
step  3000, train_accuracy = 95 % test_accuracy = 95.5 %
step  4000, train_accuracy = 98 % test_accuracy = 96.56 %
step  5000, train_accuracy = 92 % test_accuracy = 96.45 %
step  6000, train_accuracy = 96 % test_accuracy = 96.82 %
step  7000, train_accuracy = 99 % test_accuracy = 96.46 %
step  8000, train_accuracy = 95 % test_accuracy = 97.46 %
step  9000, train_accuracy = 97 % test_accuracy = 97.29 %
training time:  289.9446117  s


In [9]:
# fashion
start = time.clock()
test_X = mnist.test.images
test_X = test_X.reshape([test_size, n_steps, n_input])
test_Y = mnist.test.labels
for i in range(10000):
    batch_X, batch_Y = mnist.train.next_batch(batch_size)
    batch_X = batch_X.reshape([batch_size, n_steps, n_input])
    if i % display_step == 0:
        train_accuracy = accuracy.eval(session=sess, feed_dict={X:batch_X, Y_:batch_Y})
        test_accuracy = accuracy.eval(session=sess, feed_dict={X:test_X, Y_:test_Y})
        print("step %5d, train_accuracy = %.4g %% test_accuracy = %.4g %%" % (i, train_accuracy*100, test_accuracy*100))
    train_step.run(session=sess, feed_dict={X:batch_X, Y_:batch_Y})
print("training time: ", time.clock()-start, " s")

step     0, train_accuracy = 19 % test_accuracy = 15.97 %
step  1000, train_accuracy = 73 % test_accuracy = 73.7 %
step  2000, train_accuracy = 77 % test_accuracy = 76.35 %
step  3000, train_accuracy = 83 % test_accuracy = 77.4 %
step  4000, train_accuracy = 77 % test_accuracy = 78.06 %
step  5000, train_accuracy = 85 % test_accuracy = 82.77 %
step  6000, train_accuracy = 87 % test_accuracy = 83.76 %
step  7000, train_accuracy = 79 % test_accuracy = 84.4 %
step  8000, train_accuracy = 83 % test_accuracy = 84.82 %
step  9000, train_accuracy = 91 % test_accuracy = 85.09 %
training time:  294.1583635  s
